# XGBoost Classifier (Ensemble Model) - Income Classification
**Submission ID: 2025AA05424**

This notebook implements XGBoost Classifier (Ensemble Model) for binary classification on the Adult Income dataset.

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score, 
    matthews_corrcoef,
    confusion_matrix,
    classification_report
)
import joblib
import warnings
warnings.filterwarnings('ignore')

## 2. Install and Import XGBoost

In [ ]:
# Install XGBoost if not already installed
!pip install xgboost

from xgboost import XGBClassifier

## 3. Load Dataset

In [ ]:
# Load dataset
df = pd.read_csv("adult.csv")

print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
df.head()

## 4. Data Preprocessing

In [ ]:
# Handle missing values
df = df.replace("?", pd.NA)
print("Missing values before dropping:")
print(df.isnull().sum())

# Drop rows with missing values
df = df.dropna()
print("\nDataset shape after dropping missing values:", df.shape)

In [ ]:
# Separate features and target
X = df.drop("income", axis=1)
y = df["income"]

# Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)

print("Target variable distribution:")
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))

In [ ]:
# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)
print("Feature shape after encoding:", X.shape)
print("\nNumber of features:", X.shape[1])

## 5. Train-Test Split

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

## 6. Train XGBoost Model
**Note:** XGBoost does NOT require feature scaling

In [ ]:
# Initialize and train XGBoost model (NO SCALING)
xgb = XGBClassifier(
    n_estimators=100,
    random_state=42,
    eval_metric='logloss'
)

xgb.fit(X_train, y_train)

print("XGBoost model training completed!")
print(f"Number of boosting rounds: {xgb.n_estimators}")

## 7. Make Predictions

In [ ]:
# Make predictions
y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)[:, 1]

print("Predictions completed")
print("Sample predictions:", y_pred[:10])

## 8. Evaluate Model Performance

In [ ]:
# Calculate all 6 evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_prob)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print("\n" + "="*50)
print("XGBOOST - MODEL EVALUATION METRICS")
print("="*50)
print(f"Accuracy:  {accuracy:.4f}")
print(f"AUC Score: {auc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"MCC Score: {mcc:.4f}")
print("="*50)

In [ ]:
# Display confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

## 9. Save Model

In [ ]:
# Save model
joblib.dump(xgb, 'xgboost_model.pkl')

print("Model saved successfully!")